## Tree-based Model Feature Selection

In [7]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    roc_auc_score, roc_curve, auc, _scorer
)
from sklearn.tree import export_graphviz
import xgboost as xgb
from xgboost import XGBClassifier as xgbclass
from scipy import stats
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    make_scorer, precision_score, recall_score, f1_score,
    classification_report
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_selection import RFECV
import pickle

root_path = "../../Data/GoogleDrive/"

In [3]:
# Import pickled models
with open(root_path + "rf_model_full.pkl", "rb") as f:
    rf_model = pickle.load(f)

with open(root_path + "xgboost_model_full.pkl", "rb") as f:
    xgb_model = pickle.load(f)

In [5]:
# Load data from parquet files
X_train = pd.read_parquet(root_path + "X_train.parquet")
X_test = pd.read_parquet(root_path + "X_test.parquet")
y_train = pd.read_parquet(root_path + "y_train.parquet")
y_test = pd.read_parquet(root_path + "y_test.parquet")

### Random Forest

In [10]:
# check the models
# print(rf_model)
# print(xgb_model)

# check features
print(X_train.columns)

Index(['onehot__State_Alabama', 'onehot__State_Alaska',
       'onehot__State_Arizona', 'onehot__State_Arkansas',
       'onehot__State_California', 'onehot__State_Colorado',
       'onehot__State_Connecticut', 'onehot__State_Delaware',
       'onehot__State_District of Columbia', 'onehot__State_Florida',
       ...
       'GeneralHealth_label__GeneralHealth',
       'LastCheckupTime_label__LastCheckupTime',
       'RemovedTeeth_label__RemovedTeeth', 'SmokerStatus_label__SmokerStatus',
       'ECigaretteUsage_label__ECigaretteUsage',
       'remainder__PhysicalHealthDays', 'remainder__MentalHealthDays',
       'remainder__SleepHours', 'remainder__HeightInMeters',
       'remainder__WeightInKilograms'],
      dtype='object', length=121)


In [11]:
# Feature Selection for RF
rf_model.fit(X_train, y_train)

rf_model_rfecv = RFECV(estimator=rf_model, 
                       cv=StratifiedKFold(5), 
                       scoring='neg_mean_squared_error')

rf_model_rfecv.fit(X_train, y_train)

# check the features
print(rf_model_rfecv.support_)

# check the number of features
print(rf_model_rfecv.n_features_)

/opt/anaconda3/envs/ucla/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/ucla/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/ucla/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/ucla/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

KeyboardInterrupt: 

### XGBoost

In [19]:
best_xgb = xgb_model.best_estimator_

# Feature Selection for XGBoost
best_xgb = best_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [39]:
xgb_params = xgb_model.get_params()

Accuracy:  0.8011004737465456
Sensitivity:  0.4716739044611133
AUC:  0.8417133978217364


In [36]:
from numpy import sort
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
import pandas as pd

thresholds = sort(best_xgb.feature_importances_)

results_df = pd.DataFrame(columns=['Threshold', 'Accuracy', 'Sensitivity', 'AUC'])
print(type(results_df))

for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(best_xgb, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    
    # train model
    selection_model = XGBClassifier(**xgb_params)
    selection_model.fit(select_X_train, y_train)
    
    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    sensitivity = recall_score(y_test, predictions)
    auc = roc_auc_score(y_test, predictions)
    # save to dataframe
    # Create a DataFrame with the current results
    current_results = pd.DataFrame([[thresh, accuracy, sensitivity, auc]],
                                   columns=['Threshold', 'Accuracy', 'Sensitivity', 'AUC'])
    
    # Concatenate the current results with the existing DataFrame
    results_df = pd.concat([results_df, current_results], ignore_index=True)

<class 'pandas.core.frame.DataFrame'>


/opt/anaconda3/envs/ucla/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/opt/anaconda3/envs/ucla/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:40:29] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_b6b50u1t0e/croot/xgboost-split_1713972723244/work/cpp_src/src/learner.cc:742: 
Parameters: { "cv", "error_score", "estimator", "estimator__enable_categorical", "estimator__missing", "estimator__objective", "param_grid", "pre_dispatch", "refit", "return_train_score", "scoring", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/envs/ucla/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/var/folders/_n/xmv95vl9347b3zn8xl2bwlt40000gn/T/ipykernel_26695/2865814613.py:34: FutureWarning: The behavior of DataFrame concatenation wit

In [40]:
# print accuracy, sensitivity, AUC for xgb_model
y_pred_xgb = best_xgb.predict(X_test)
y_pred_proba_xgb = best_xgb.predict_proba(X_test)[:, 1]

print("Accuracy: ", accuracy_score(y_test, y_pred_xgb))
print("Sensitivity: ", recall_score(y_test, y_pred_xgb))
print("AUC: ", roc_auc_score(y_test, y_pred_proba_xgb))

# print top 5 results with the best sensitivity
print(results_df.sort_values(by='Sensitivity', ascending=False).head(5))

Accuracy:  0.8011004737465456
Sensitivity:  0.4716739044611133
AUC:  0.8417133978217364
    Threshold  Accuracy  Sensitivity       AUC
87   0.003053  0.799472     0.481741  0.693562
72   0.002371  0.798658     0.480853  0.692723
76   0.002634  0.798781     0.480359  0.692640
90   0.003235  0.798238     0.479866  0.692114
86   0.002957  0.798954     0.479372  0.692427
